### Hash Functions

This section consists of toy hash functions and the cr game. 

In [1]:
from tools.utils import xor
from game.cr import *
from simulate import *
from tools.block_cipher import *

In [2]:
def H1(K, x):
	return int(x) % int(K)


In [3]:
E = BlockCipher(key_len = 5, block_len= 5)

def H2(K, x):
	'''
	Let E: {0,1}^n * {0,1}^n -> {0,1}^n be a block cipher. 
	Let H: {0,1}^k * {0,1}^2n -> {0,1}^n be a hash function. 
	'''

	key_len, block_len, message_len = 5, 5, 2*5
	assert len(x) == message_len, f"Expected {message_len} got {len(x)} for messages"
	
	y = E.evaluate(K, xor(E.evaluate(K, x[0:block_len]), x[block_len:]))
	return y

## Define the CR against H2 with key length 5
game = CR(H2, 5)

## Initializing the game. Note the adversary has access to the key k. 
k = game._initialize()

def adv_H2(game):
	'''
	This is the adversary against H2.
	Define and efficient adversary that ouptuts two messages x1 and x1 such that H(x1) = H(x2)

	Note: Ensure that you use strings of 0s and 1s as input. 
	Eg x1 = '00000'
		x2 = '10101'

	Hint: As the adversary has access to the key, the adversary can compute E_k and E_k^{-1}
	E_k(x) = E.block_cipher(k, x) and
	E_k^{-1}(c) = E.block_cipher_inverse(k, c)

	'''	
	#========= Your Code goes here =======================
	x_1 = '0'*5 + '1'*5
	x2_2 = '0'*5

	x2_1 = E.inverse(k, xor(xor(E.evaluate(k, x_1[0:5]), x_1[5:]), x2_2))
	x_2 = x2_1 + x2_2
	# =====================================================
	return x_1, x_2

x1, x2 = adv_H2(game)

print(game.finalize(x1, x2))

True


### Keyless Hash Functions
We say that H: Keys * D -> R is keyless if Keys = {$\epsilon$}consists of just one key, the empty string.

In [4]:
b = 5
n = 7

E = BlockCipher(key_len=b, block_len=n)

def H3(K, x):
	'''
	Let E: {0,1}^n * {0,1}^n -> {0,1}^n be a block cipher and 
	Let Let H: {0,1}^b+n  -> {0,1}^n be a keyless hash function defined as h(x||v) == E_x(v)
	'''

	key = x[0:b]
	v = x[b:]
	
	assert len(key) == b 
	assert len(v) == n

	return E.evaluate(key, v)

game = CR(H3, key_len=b)
k = game._initialize()

def adv_H3(game):
	x1 = '0'*b
	x2 = '1'*b
	v1 = '0'*n
	y = E.evaluate(key=x1, plaintext=v1)
	v2 = E.inverse(key= x2, ciphertext=y)
	return x1+v1, x2+v2

x1, x2 = adv_H3(game)
print(game.finalize(x1,x2))

True


### Homework 4 Problem 1

In [6]:
from tools.AES import *

key_len = 128
output_len = 128
aes = AES(key_len, block_len=128)

def H_K(K, M):
	c_0 = '0'*128
	m = len(M)// key_len
	assert m*key_len == len(M), "Message length has to be a multiple of 128"

	for i in range(m):
		B_i = aes.evaluate(K, xor(c_0, M[i*len(K): (i+1)*len(K)]))
		C_i = aes.evaluate(K, xor(B_i, M[i*len(K): (i+1)*len(K)]))
		c_0 = C_i
	return C_i

game = CR(H_K, key_len = 128)


def adv_H_K(game):
	key = game._initialize()

	m1 = aes.inverse( key, '0'*128)
	m2 = aes.inverse( key, '0'*128) + aes.inverse(key, '0'*128)
	return m1, m2
x1, x2 = adv_H_K(game)

# print(game.finalize(x1, x2))

s = Simulate(game, adv_H_K)
s.simulate_cr(verbose=True)

Pr[CR_H => 1] = 1000 / 1000 = 1.0

Adv(A) = 1.0
